In [26]:
import copy
import pandas as pd
import numpy as np
import torch as T
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from datetime import datetime
device = T.device("cuda" if T.cuda.is_available() else "cpu")

In [2]:
cycle_data = pd.read_csv('../data/raw_cycle_labeled.csv', index_col='timestamp')
cycle_data.head()

,current,voltage,temperature,cycle
timestamp,,,,
1585699584,-0.001839,0.961446,0.941176,0
1585699585,-0.016636,0.960964,0.941176,0
1585699586,-0.037809,0.960482,0.941176,0
1585699587,-0.088862,0.959277,0.941176,0
1585699588,-0.074760,0.959036,0.941176,0


In [3]:
print(f"num cycles: {cycle_data.cycle.unique().max()}")
cycle_records = []
for cycle in cycle_data.cycle.unique():
    data = cycle_data.loc[cycle_data.cycle == cycle]
    record = {}
    record['cycle'] = cycle
    record['voltage'] = data.voltage.astype(np.float32).values
    record['current'] = data.current.astype(np.float32).values
    record['temperature'] = data.temperature.astype(np.float32).values
    cycle_records.append(record)
cycles = pd.DataFrame.from_records(cycle_records)
cycles.set_index('cycle')
cycles.head()

num cycles: 22


,cycle,voltage,current,temperature
0,0,"[0.9614458, 0.96096385, 0.96048194, 0.9592771,...","[-0.0018393623, -0.01663601, -0.037809115, -0....","[0.9411765, 0.9411765, 0.9411765, 0.9411765, 0..."
1,1,"[0.9592771, 0.9592771, 0.9592771, 0.9592771, 0...","[0.5085633, 0.50063354, 0.49593297, 0.49646434...","[0.9485294, 0.9485294, 0.9485294, 0.9485294, 0..."
2,2,"[0.9696385, 0.9696385, 0.9696385, 0.9696385, 0...","[0.51162887, 0.511588, 0.50991213, 0.5027999, ...","[0.94485295, 0.94485295, 0.94485295, 0.9448529..."
3,3,"[0.97927713, 0.97951806, 0.97951806, 0.9792771...","[0.50647867, 0.5064378, 0.50247294, 0.49777234...","[0.94485295, 0.94485295, 0.94485295, 0.9448529..."
4,4,"[0.9891566, 0.9891566, 0.9891566, 0.9891566, 0...","[0.5014919, 0.5014511, 0.49642345, 0.49286735,...","[0.9411765, 0.9411765, 0.9411765, 0.9411765, 0..."


In [21]:
def create_padded_tensor(df, column_name):
    sequences = [T.tensor(seq).unsqueeze(1).float() for seq in df[column_name]]
    dataset = pad_sequence(sequences, padding_value=0, batch_first=True)
    n_seq, seq_len, n_features = dataset.shape
    return dataset, seq_len, n_seq, n_features
feature_dataset, seq_len, n_seq, n_features = create_padded_tensor(cycles, 'current')
train_X, val_X, test_X = feature_dataset[0:15], feature_dataset[15:19], feature_dataset[19:]
print(f"seq_len:{seq_len}, n_seq:{n_seq}, n_features:{n_features}")
target_dataset, _, _, _ = create_padded_tensor(cycles, 'voltage')
train_Y, val_Y, test_Y = target_dataset[0:15], target_dataset[15:19], target_dataset[19:]

seq_len:48711, n_seq:23, n_features:1


In [9]:
class Encoder(T.nn.Module):

  def __init__(self, seq_len, n_features, embedding_dim=64):
    super(Encoder, self).__init__()

    self.seq_len, self.n_features = seq_len, n_features
    self.embedding_dim, self.hidden_dim = embedding_dim, 2 * embedding_dim

    self.rnn1 = T.nn.LSTM(
      input_size=n_features,
      hidden_size=self.hidden_dim,
      num_layers=1,
      batch_first=True
    )
    
    self.rnn2 = T.nn.LSTM(
      input_size=self.hidden_dim,
      hidden_size=embedding_dim,
      num_layers=1,
      batch_first=True
    )

  def forward(self, x):
    x = x.reshape((1, self.seq_len, self.n_features))

    x, (_, _) = self.rnn1(x)
    x, (hidden_n, _) = self.rnn2(x)

    return hidden_n.reshape((self.n_features, self.embedding_dim))

In [10]:
class Decoder(T.nn.Module):

  def __init__(self, seq_len, input_dim=64, n_features=1):
    super(Decoder, self).__init__()

    self.seq_len, self.input_dim = seq_len, input_dim
    self.hidden_dim, self.n_features = 2 * input_dim, n_features

    self.rnn1 = T.nn.LSTM(
      input_size=input_dim,
      hidden_size=input_dim,
      num_layers=1,
      batch_first=True
    )

    self.rnn2 = T.nn.LSTM(
      input_size=input_dim,
      hidden_size=self.hidden_dim,
      num_layers=1,
      batch_first=True
    )

    self.output_layer = T.nn.Linear(self.hidden_dim, n_features)

  def forward(self, x):
    x = x.repeat(self.seq_len, self.n_features)
    x = x.reshape((self.n_features, self.seq_len, self.input_dim))

    x, (hidden_n, cell_n) = self.rnn1(x)
    x, (hidden_n, cell_n) = self.rnn2(x)
    x = x.reshape((self.seq_len, self.hidden_dim))

    return self.output_layer(x)

In [11]:
class RecurrentAutoencoder(T.nn.Module):

  def __init__(self, seq_len, n_features, embedding_dim=64):
    super(RecurrentAutoencoder, self).__init__()

    self.encoder = Encoder(seq_len, n_features, embedding_dim).to(device)
    self.decoder = Decoder(seq_len, embedding_dim, n_features).to(device)

  def forward(self, x):
    x = self.encoder(x)
    x = self.decoder(x)

    return x

In [22]:
model = RecurrentAutoencoder(seq_len, n_features, 128)
model = model.to(device)

In [27]:
def train_model(model, train_dataset, val_dataset, n_epochs):
  optimizer = T.optim.Adam(model.parameters(), lr=1e-3)
  criterion = T.nn.L1Loss(reduction='sum').to(device)
  history = dict(train=[], val=[])

  best_model_wts = copy.deepcopy(model.state_dict())
  best_loss = 10000000.0
  
  for epoch in range(1, n_epochs + 1):
    start = datetime.now()
    model = model.train()

    train_losses = []
    for seq_true in train_dataset:
      optimizer.zero_grad()

      seq_true = seq_true.to(device)
      seq_pred = model(seq_true)

      loss = criterion(seq_pred, seq_true)

      loss.backward()
      optimizer.step()

      train_losses.append(loss.item())

    val_losses = []
    model = model.eval()
    with T.no_grad():
      for seq_true in val_dataset:

        seq_true = seq_true.to(device)
        seq_pred = model(seq_true)

        loss = criterion(seq_pred, seq_true)
        val_losses.append(loss.item())

    train_loss = np.mean(train_losses)
    val_loss = np.mean(val_losses)

    history['train'].append(train_loss)
    history['val'].append(val_loss)

    if val_loss < best_loss:
      best_loss = val_loss
      best_model_wts = copy.deepcopy(model.state_dict())
    end = datetime.now()
    print(f'Epoch {epoch}: train loss {train_loss} val loss {val_loss}. Duration {end-start}')

  model.load_state_dict(best_model_wts)
  return model.eval(), history

In [28]:
model, history = train_model(
  model, 
  train_X, 
  val_X, 
  n_epochs=5
)

Epoch 1: train loss 1032.047762044271 val loss 734.994140625. Duration 0:20:32.127490
Epoch 2: train loss 749.9232757568359 val loss 714.5650329589844. Duration 0:25:13.780087
Epoch 3: train loss 676.8711395263672 val loss 751.3839874267578. Duration 0:22:58.696246
Epoch 4: train loss 685.8586771647135 val loss 699.7767486572266. Duration 0:22:05.844458
Epoch 5: train loss 672.9889862060547 val loss 718.4171905517578. Duration 0:24:20.604843
